In [1]:
# 1. Mount Drive and load preprocessed data
from google.colab import drive
import os
import numpy as np

drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/spacecraft_anomaly_project'
PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')

X = np.load(os.path.join(PROCESSED_DIR, 'X_windows.npy'))
y = np.load(os.path.join(PROCESSED_DIR, 'y_windows.npy'))  # Only for evaluation

print(f"X shape: {X.shape} | y shape: {y.shape}")

Mounted at /content/drive
X shape: (2094, 30, 21) | y shape: (2094,)


In [2]:
# Only use normal windows for training
X_train = X[y == 0]
print(f"Training on {X_train.shape[0]} normal windows")

Training on 365 normal windows


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.losses import MeanSquaredError

timesteps = X.shape[1]
n_features = X.shape[2]

def build_lstm_autoencoder(timesteps, n_features):
    input_layer = layers.Input(shape=(timesteps, n_features))

    # Encoder
    x = layers.LSTM(64, return_sequences=True)(input_layer)
    x = layers.LSTM(32, return_sequences=False)(x)

    # Bottleneck
    x = layers.RepeatVector(timesteps)(x)

    # Decoder
    x = layers.LSTM(32, return_sequences=True)(x)
    x = layers.LSTM(64, return_sequences=True)(x)
    output_layer = layers.TimeDistributed(layers.Dense(n_features))(x)

    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss=MeanSquaredError())
    return model

model = build_lstm_autoencoder(timesteps, n_features)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 30, 21)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 30, 64)         │        22,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector_1 (RepeatVector)  │ (None, 30, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 30, 32)         │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 30, 64)         │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 30, 21)         │         1,365 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 68,949 (269.33 KB)

 Trainable params: 68,949 (269.33 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, X_train,
    epochs=50,
    batch_size=64,
    validation_split=0.1,
    callbacks=[early_stop],
    shuffle=True
)

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 14s 377ms/step - loss: 0.7039 - val_loss: 0.3749
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.4991 - val_loss: 0.2503
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 0.3893 - val_loss: 0.2056
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - loss: 0.3230 - val_loss: 0.1849
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.2999 - val_loss: 0.1815
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 0.2978 - val_loss: 0.1724
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 0.2738 - val_loss: 0.1669
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - loss: 0.2717 - val_loss: 0.1633
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - loss: 0.2527 - val_loss: 0.1607
Epoch 10/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - loss: 0.2486 - val_loss: 0.1595
Epoch 11/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - loss: 0.2349 - val_loss: 0.1580
Epoch 12/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - loss: 0.2456 - val_loss: 0.1

In [7]:
MODEL_DIR = os.path.join(BASE_DIR, 'models', 'autoencoder')
os.makedirs(MODEL_DIR, exist_ok=True)

model.save(os.path.join(MODEL_DIR, 'lstm_autoencoder.h5'))
print("✅ Model saved.")

✅ Model saved.
